In [ ]:
%reset -f

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [ ]:
import cv2

def properties_video(url):
  # Opens the Video file
  video = cv2.VideoCapture(url)
  frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
  fps = int(video.get(cv2.CAP_PROP_FPS))
  width  = video.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
  height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
  duration = frame_count/fps
  minutes = int(duration/60)
  seconds = duration%60
  print("Total de frames: " + str(int(frame_count)))
  print("FPS: " + str(fps) )
  print('Duration (M:S) = ' + str(minutes) + ':' + str(seconds))
  print('Width, Height:', width, height)

In [ ]:
properties_video("/content/gdrive/MyDrive/Remasterizar/habana_split_stabilized.mp4")

In [ ]:
################# Required Configurations ############################

#Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder,
#the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
INPUT_FILEPATH = "Remasterizar/habana_split_stabilized.mp4" 

#Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is
#recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "Remasterizar/habana_split_dain.mp4" 

# Target FPS en este caso seria los FPS del video original * la cantidad de fluidez
TARGET_FPS = 232 

#A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/DAIN/input_frames' 

#A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames' 

#First frame to consider from the video when processing.
START_FRAME = 1 

#Last frame to consider from the video when processing. To use the whole video use `-1`.
END_FRAME = -1 

#Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False 

#This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem,
#but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and
#cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True 

In [ ]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi

In [ ]:
# Install known used versions of PyTorch and SciPy
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.1.0

!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Finished installing dependencies.")

In [ ]:
# Clone DAIN sources
%cd /content
!git clone -b master --depth 1 https://github.com/baowenbo/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

In [ ]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")

In [ ]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

In [ ]:
# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [ ]:
# Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Input file has {int(fps)} fps")

frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(f"Input file has {int(frame_count)} frames")

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [ ]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

if (END_FRAME==-1):
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=gte(n\,{START_FRAME}),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'
else:
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=between(n\,{START_FRAME}\,{END_FRAME}),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'

from IPython.display import clear_output
clear_output()

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
frame_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS:
  frame_count += 1
  first_frame = f"{FRAME_INPUT_DIR}/00001.png"
  new_last_frame = f"{FRAME_INPUT_DIR}/{frame_count.zfill(5)}.png"
  shutil.copyfile(first_frame, new_last_frame)

print(f"{frame_count} frame PNGs generated.")

In [ ]:
# Checking if PNGs do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha channel detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

In [ ]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

In [ ]:
# Finding DAIN Frames, upscaling and cropping to match original
%cd {FRAME_OUTPUT_DIR}

if (RESIZE_HOTFIX):
  images = []
  for filename in os.listdir(FRAME_OUTPUT_DIR):
    img = cv2.imread(os.path.join(FRAME_OUTPUT_DIR, filename))
    filename = os.path.splitext(filename)[0]
    if(not filename.endswith('0')):
      dimensions = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimensions, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimensions[1]-1), 1:(dimensions[0]-1)]
      cv2.imwrite(f"{filename}.png", crop)

In [ ]:
# Create output video
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

In [ ]:
!cp -r /content/DAIN/output_frames /content/gdrive/MyDrive/Remasterizar/output_frames_dain

In [ ]:
import os

path, dirs, files = next(os.walk("/content/DAIN/output_frames"))
print("Hay " + str(len(files)) + " en -> " + str("/content/DAIN/output_frames"))

path, dirs, files = next(os.walk("/content/DAIN/output_frames"))
print("Hay " + str(len(files)) + " en -> " + str("/content/gdrive/MyDrive/Remasterizar/output_frames"))